1. had to change chains to 2 when compiling the models

In [32]:
outputfile <- 'stan-fit-US.rdata'
samplesfile <- 'samples-US.rdata'
resultsfile <- 'results-elites-US.rdata'

In [59]:
source('Barbera/functions.R')

## change the following lines to run this Rscript for other countries
matrixfile <- 'Barbera/adj-matrix-US.rdata'
#outputfile <- 'temp/stan-fit-US.rdata'
#samplesfile <- 'output/samples-US.rdata'
#resultsfile <- 'output/results-elites-US.rdata'
country <- 'US'

# parameters for Stan model
n.iter <- 500
n.warmup <- 100
thin <- 2 ## this will give up to 200 effective samples for each chain and par

# loading data
load(matrixfile)

## starting values for elites (for identification purposes)
load("Barbera/elites-data.Rdata")

# US:
if (country=="US"){
	us <- elites.data[['US']]
	parties <- merge(
		data.frame(screen_name = colnames(y), stringsAsFactors=F),
		us[,c("screen_name", "party")], sort=FALSE, all.x=TRUE)$party
	start.phi <- rep(0, length(parties))
	start.phi[parties=="D"] <- -1
	start.phi[parties=="R"] <- 1
}

J <- dim(y)[1]

# choosing a sample of 10,000 "informative" users who follow 10 or more
# politicians, and then subsetting politicians followed by >200 of these

if (J>100){
  J <- 100
  inform <- which(rowSums(y)>10)
  set.seed(12345)
  subset.i <- sample(inform, J)
  y <- y[subset.i, ]
  start.phi <- start.phi[which(colSums(y)>2)]
  y <- y[,which(colSums(y)>2)]
}

## data for model
J <- dim(y)[1]
K <- dim(y)[2]
N <- J * K
jj <- rep(1:J, times=K)
kk <- rep(1:K, each=J)

stan.data <- list(J=J, K=K, N=N, jj=jj, kk=kk, y=c(as.matrix(y)))

## rest of starting values
colK <- colSums(y)
rowJ <- rowSums(y)
normalize <- function(x){ (x-mean(x))/sd(x) }

# set the initial parameters for the model. Currently set at 2 sets of inits. 1 init = 1 chain
inits <- rep(list(list(alpha=normalize(log(colK+0.0001)), 
	beta=normalize(log(rowJ+0.0001)),
  theta=rnorm(J), phi=start.phi,mu_beta=0, sigma_beta=1, 
  gamma=abs(rnorm(1)), mu_phi=0, sigma_phi=1, sigma_alpha=1)),2)


In [35]:
library(rstan)

In [60]:
#bilinear
stan.code <- '
data {
  int<lower=1> J; // number of twitter users
  int<lower=1> K; // number of elite twitter accounts
  int<lower=1> N; // N = J x K
  int<lower=1,upper=J> jj[N]; // twitter user for observation n
  int<lower=1,upper=K> kk[N]; // elite account for observation n
  int<lower=0,upper=1> y[N]; // dummy if user i follows elite j
}
parameters {
  vector[K] alpha;
  vector[K] phi;
  vector[J] theta;
  vector[J] beta;
  real mu_beta;
  real<lower=0.1> sigma_beta;
  real mu_phi;
  real<lower=0.1> sigma_phi;
  real<lower=0.1> sigma_alpha;
  real gamma;
}
model {
  alpha ~ normal(0, sigma_alpha);
  beta ~ normal(mu_beta, sigma_beta);
  phi ~ normal(mu_phi, sigma_phi);
  theta ~ normal(0, 1); 
  for (n in 1:N)
    y[n] ~ bernoulli_logit( alpha[kk[n]] + beta[jj[n]] + 
      gamma * ( theta[jj[n]] * phi[kk[n]] ) );
}
'

## compiling model
stan.model <- stan(model_code=stan.code, 
    data = stan.data, init=inits, iter=1, warmup=0, chains=2)


SAMPLING FOR MODEL '56c1f1475e9520486c332f520fc23023' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.002722 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 27.22 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: WARNING: No variance estimation is
Chain 1:          performed for num_warmup < 20
Chain 1: 
Chain 1: Iteration: 1 / 1 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1e-06 seconds (Warm-up)
Chain 1:                0.003141 seconds (Sampling)
Chain 1:                0.003142 seconds (Total)
Chain 1: 

SAMPLING FOR MODEL '56c1f1475e9520486c332f520fc23023' NOW (CHAIN 2).
Chain 2: 
Chain 2: Gradient evaluation took 0.001792 seconds
Chain 2: 1000 transitions using 10 leapfrog steps per transition would take 17.92 seconds.
Chain 2: Adjust your expectations accordingly!
Chain 2: 
Chain 2: 
Chain 2: WARNING: No variance estimation is
Chain 2:          performed for num_warmup < 20
Chain 2: 
Chai

Warning message:
“There were 2 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is NA, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


In [63]:
# run bilinear with benchmark
start <- Sys.time()

stan.fit <- stan(fit=stan.model, data = stan.data, 
	iter=n.iter, warmup=n.warmup, chains=2, 
  	thin=thin, cores=2)

end <- Sys.time()
end - start

Warning message:
“There were 156 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 2.24, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


Time difference of 3.322069 mins

In [64]:
# euclidean
stan.code <- '
data {
  int<lower=1> J; // number of twitter users
  int<lower=1> K; // number of elite twitter accounts
  int<lower=1> N; // N = J x K
  int<lower=1,upper=J> jj[N]; // twitter user for observation n
  int<lower=1,upper=K> kk[N]; // elite account for observation n
  int<lower=0,upper=1> y[N]; // dummy if user i follows elite j
}
parameters {
  vector[K] alpha;
  vector[K] phi;
  vector[J] theta;
  vector[J] beta;
  real mu_beta;
  real<lower=0.1> sigma_beta;
  real mu_phi;
  real<lower=0.1> sigma_phi;
  real<lower=0.1> sigma_alpha;
  real gamma;
}
model {
  alpha ~ normal(0, sigma_alpha);
  beta ~ normal(mu_beta, sigma_beta);
  phi ~ normal(mu_phi, sigma_phi);
  theta ~ normal(0, 1); 
  for (n in 1:N)
    y[n] ~ bernoulli_logit( alpha[kk[n]] + beta[jj[n]] - 
      gamma * square( theta[jj[n]] - phi[kk[n]] ) );
}
'

## compiling model
stan.model <- stan(model_code=stan.code, 
    data = stan.data, init=inits, iter=1, warmup=0, chains=2)


SAMPLING FOR MODEL '8992c9a87bed3e79f326f940366064d3' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.003689 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 36.89 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: WARNING: No variance estimation is
Chain 1:          performed for num_warmup < 20
Chain 1: 
Chain 1: Iteration: 1 / 1 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 2e-06 seconds (Warm-up)
Chain 1:                0.00436 seconds (Sampling)
Chain 1:                0.004362 seconds (Total)
Chain 1: 

SAMPLING FOR MODEL '8992c9a87bed3e79f326f940366064d3' NOW (CHAIN 2).
Chain 2: 
Chain 2: Gradient evaluation took 0.002426 seconds
Chain 2: 1000 transitions using 10 leapfrog steps per transition would take 24.26 seconds.
Chain 2: Adjust your expectations accordingly!
Chain 2: 
Chain 2: 
Chain 2: WARNING: No variance estimation is
Chain 2:          performed for num_warmup < 20
Chain 2: 
Chain

Warning message:
“There were 2 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is NA, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


In [65]:
## running modle
# run euclidean with benchmark
# removed inits=inits
start <- Sys.time()

stan.fit <- stan(fit=stan.model, data = stan.data, 
	iter=n.iter, warmup=n.warmup, chains=2, 
  	thin=thin, cores=2)

end <- Sys.time()
end - start

Warning message:
“The largest R-hat is 1.87, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


Time difference of 1.822793 mins

In [56]:
samples <- extract(stan.fit, pars=c("alpha", "phi", "gamma", "mu_beta",
	"sigma_beta", "sigma_alpha"))

#save(samples, file=samplesfile)

In [57]:
results <- data.frame(
#	screen_name = samples$m.names,
	phi = apply(samples$phi, 2, mean),
	phi.sd = apply(samples$phi, 2, sd),
	alpha = apply(samples$alpha, 2, mean),
	alpha.sd = apply(samples$alpha, 2, sd),
	stringsAsFactors=F)
#save(results, file=resultsfile)

In [58]:
results

phi,phi.sd,alpha,alpha.sd
<dbl>,<dbl>,<dbl>,<dbl>
-1.84434938,3.3745556,0.47309734,4.7889559
-0.74885648,1.6178200,1.12970098,1.5134880
0.05421408,0.3811940,-2.07698048,0.5834708
-0.97974968,2.2575204,-1.02013284,2.3715283
-1.88055166,3.6438411,-1.18374792,5.5505241
1.40394413,2.6136077,0.11914662,3.0526485
1.22119120,2.2111835,1.57708774,2.3674684
-1.46933863,3.1010245,-1.49369684,4.1113335
-0.63477240,1.5535769,-1.44700556,1.3935652
